# PyTorch Techniques for Model Optimization

## Table of Contents

[PyTorch Techniques for Model Optimization](#pytorch-techniques-for-model-optimization)

- [Saving Progress w/ Model Checkpointing](##saving-progress-w-model-checkpointing)

## Saving Progress w/ Model Checkpointing

**Model Checkpointing Intro.** - Will now focus on model checkpointing using PyTorch. This is vital technique in machine learning that allows save state of model during training, ensuring best-performing models are preserved. Will come to understand how implement model checkpointing, allowing to save model whenever achieves best performance on a validation set.

So model checkpointing involves saving state of a neural network model at various points during training process. Crucial for several reasons:
- **Prevent Loss of Progress**: In case of unexpected interruptions (e.g., power failure, hardware consumption), checkpointing helps resuming training from last saved state.
- **Save Best Performing Models**: By saving model whenever achieves a new best performance on validation set, ensure that retain best version of our model.

**Setting up Environment** - Assume set up environment seen before and used below: import necessary libraries, do preprocessing of Wine dataset, define model, define loss and optimizer. Will for now omit training loop with eval, graphing of loss and finally saving model loading and confirming same val_loss, as this code will be modified with checkpointing.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import sklearn.utils as skUtils

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Tuple, Dict

wine_set: skUtils.Bunch = load_wine()

def load_preprocessed_data(wine: skUtils.Bunch) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    X: np.ndarray = wine.data
    y: np.ndarray = wine.target

    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, stratify=y)

    scaler: StandardScaler = StandardScaler().fit(Xtrain)
    Xtrain_scaled: np.ndarray = scaler.transform(Xtrain)
    Xtest_scaled: np.ndarray = scaler.transform(Xtest)

    Xtrain_tensor: torch.Tensor = torch.tensor(Xtrain_scaled, dtype=torch.float32)
    Xtest_tensor: torch.Tensor = torch.tensor(Xtest_scaled, dtype=torch.float32)
    ytrain_tensor: torch.Tensor = torch.tensor(ytrain, dtype=torch.long)
    ytest_tensor: torch.Tensor = torch.tensor(ytest, dtype=torch.long)

    return Xtrain_tensor, Xtest_tensor, ytrain_tensor, ytest_tensor


X_train, X_test, y_train, y_test = load_preprocessed_data(wine_set)

model: nn.Sequential = nn.Sequential( # MAKE SURE YOU CAN CUSTOM DEFINE TOO!!
    nn.Linear(in_features=13, out_features=10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 3)
)

criterion: nn.CrossEntropyLoss = nn.CrossEntropyLoss()
optimizer: optim.Adam = optim.Adam(model.parameters(), lr=0.001)